In [11]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [1]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('Database/S&P500.db')
query = "SELECT * FROM stock"
stock = pd.read_sql_query(query, conn)
ticker_by_sector = stock.groupby(by='gics_sector')['symbol'].apply(list).to_dict()

conn.close()

In [2]:
import pandas as pd

def expand_to_5minutes(row, columns):
    dates = pd.date_range(
        start=row.name + pd.Timedelta(hours=14, minutes=30),
        end=row.name + pd.Timedelta(hours=20, minutes=55),
        freq='5min'
    )
    expanded_rows = pd.DataFrame(index=dates, columns=columns)
    expanded_rows.index.name = 'Date'
    for col in columns:
        expanded_rows[col] = row[col]
    return expanded_rows

In [ ]:
import pandas as pd
import yfinance as yf

for sector, tickers in ticker_by_sector.items():
    for ticker in tickers:
        '''
        일일 데이터 가공
        '''
        # 지표들을 계산할 수 있는 최소한의 데이터 불러오기
        start = pd.to_datetime('now') - pd.Timedelta(days=444)
        end = pd.to_datetime('now') - pd.Timedelta(days=59)
        daily_data = yf.download(tickers=ticker, start=f"{start.year}-{start.month}-{start.day}", end=f"{end.year}-{end.month}-{end.day}", interval='1d')
        if daily_data.empty:
            continue
        # 데이터 5분 단위로 확장
        expanded_data = pd.concat([expand_to_5minutes(row, daily_data.columns) for _, row in daily_data.iterrows()])
        # 5분 단위의 거래량을 평균으로 맞춤
        expanded_data['Volume'] = (expanded_data['Volume'] / 78).astype(int)
        
        
        '''
        5분 단위 데이터 가공
        '''
        # 5분 단위 데이터를 최대한 불러오기
        start = pd.to_datetime('now') - pd.Timedelta(days=59)
        end = pd.to_datetime('now')
        intraday_data = yf.download(tickers=ticker, start=f"{start.year}-{start.month}-{start.day}", end=f"{end.year}-{end.month}-{end.day}", interval='5m')
        if intraday_data.empty:
            continue
        
        intraday_data = intraday_data.rename_axis('Date')
        intraday_data.index = intraday_data.index.tz_localize(None)
        
        
        '''
        데이터 병합
        '''
        data = pd.concat([expanded_data, intraday_data])
        data.index = pd.to_datetime(data.index)
        
        
        '''
        고가, 저가 수정
        '''
        data['High'] = data['High'].rolling(window=78).max()
        data['Low'] = data['Low'].rolling(window=78).min()
        
        
        '''
        결측치 삭제
        '''
        data = data.dropna()
        
        
        '''
        csv 파일로 저장
        '''
        data.to_csv(f'Data/Price/{ticker}_price_5min.csv', index=True)
        print(f'{ticker} data saved')
        

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


AAPL data saved


In [8]:
'''
저장한 데이터 확인
'''
import pandas as pd

read_data = pd.read_csv(f'Data/AAPL_price_5min.csv', header=[0, 1], index_col=0)
display(read_data)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2023-10-12 14:30:00,179.592422,180.710007,182.339996,179.039993,180.070007,56743100.0
2023-10-12 14:35:00,179.592422,180.710007,182.339996,179.039993,180.070007,56743100.0
2023-10-12 14:40:00,179.592422,180.710007,182.339996,179.039993,180.070007,56743100.0
2023-10-12 14:45:00,179.592422,180.710007,182.339996,179.039993,180.070007,56743100.0
2023-10-12 14:50:00,179.592422,180.710007,182.339996,179.039993,180.070007,56743100.0
...,...,...,...,...,...,...
2024-12-24 17:20:00,257.349701,257.349701,257.540009,257.329987,257.359985,194459.0
2024-12-24 17:25:00,257.359985,257.359985,257.440002,257.269989,257.341705,176048.0


In [21]:
'''
데이터 업데이트
'''
import pandas as pd
import yfinance as yf

for sector, tickers in ticker_by_sector.items():
    for ticker in tickers:
        # 기존 데이터 불러오기
        try:
            data = pd.read_csv(f'Data/Price/{ticker}_price_5min.csv', header=[0, 1], index_col=0)
        except FileNotFoundError:
            continue
        
        # 데이터 정리
        # data = data.xs(ticker, level=1, axis=1)
        data.index = pd.to_datetime(data.index)
        data = data.sort_index(ascending=True)

        # 최근 보름 5분 단위 가격 데이터 불러오기
        start = pd.to_datetime('now') - pd.Timedelta(days=15)
        end = pd.to_datetime('now')
        intraday_data = yf.download(tickers=ticker, start=f"{start.year}-{start.month}-{start.day}", end=f"{end.year}-{end.month}-{end.day}", interval='5m')
        if intraday_data.empty:
            continue
        intraday_data = intraday_data.rename_axis('Date')
        intraday_data.index = intraday_data.index.tz_localize(None)
        
        # 중복된 인덱스를 제외한 새로운 데이터 병합
        data = pd.concat([data, intraday_data[~intraday_data.index.isin(data.index)]])
        data = data.sort_index(ascending=True)
        
        # 데이터 저장
        data.to_csv(f"Data/Price/{ticker}_price_5min.csv", index=True)
        print(f'{ticker} data updated')

[*********************100%***********************]  1 of 1 completed


GOOGL data updated


[*********************100%***********************]  1 of 1 completed


GOOG data updated


[*********************100%***********************]  1 of 1 completed


...

VST data updated


[*********************100%***********************]  1 of 1 completed


WEC data updated


[*********************100%***********************]  1 of 1 completed


XEL data updated


In [87]:
'''
기존 데이터 수정
'''
import pandas as pd
import yfinance as yf

for sector, tickers in ticker_by_sector.items():
    for ticker in tickers:
        # 기존 데이터 불러오기
        try:
            data = pd.read_csv(f'Data/Price/{ticker}_price_5min.csv', header=[0, 1], index_col=0)
        except FileNotFoundError:
            continue
        
        # 데이터 정리
        # data = data.xs(ticker, level=1, axis=1)
        data.index = pd.to_datetime(data.index)
        data = data.sort_index(ascending=True)
        
        # 일 단위 데이터 거래량 수정(이미 수정이 되어있으면 주석처리 할 것)
        data.loc[:'2024-10-25', ('Volume', ticker)] = (data.loc[:'2024-10-25', ('Volume', ticker)] / 78).astype(int)
        
        # 5분 단위 데이터 고가, 저가 수정
        data.loc['2024-10-26':, ('Low', ticker)] = data.loc['2024-10-25':, ('Close', ticker)].rolling(window=78).min().loc['2024-10-26':]
        data.loc['2024-10-26':, ('High', ticker)] = data.loc['2024-10-25':, ('Close', ticker)].rolling(window=78).max().loc['2024-10-26':]
        
         # 데이터 저장
        data.to_csv(f"Data/Price/Fix/{ticker}_price_5min.csv", index=True)
        print(f'{ticker} data fixed')

GOOGL data fixed
GOOG data fixed
T data fixed
CHTR data fixed
CMCSA data fixed
EA data fixed
FOXA data fixed
FOX data fixed
IPG data fixed
LYV data fixed
MTCH data fixed
META data fixed
NFLX data fixed
NWSA data fixed
NWS data fixed
OMC data fixed
PARA data fixed
TMUS data fixed
TTWO data fixed
VZ data fixed
DIS data fixed
WBD data fixed
ABNB data fixed
AMZN data fixed
APTV data fixed
AZO data fixed
BBY data fixed
BKNG data fixed
BWA data fixed
CZR data fixed
KMX data fixed
CCL data fixed
CMG data fixed
DRI data fixed
DECK data fixed
DPZ data fixed
DHI data fixed
EBAY data fixed
EXPE data fixed
F data fixed
GRMN data fixed
GM data fixed
GPC data fixed
HAS data fixed
HLT data fixed
HD data fixed
LVS data fixed
LEN data fixed
LKQ data fixed
LOW data fixed
LULU data fixed
MAR data fixed
MCD data fixed
MGM data fixed
MHK data fixed
NKE data fixed
NCLH data fixed
NVR data fixed
ORLY data fixed
POOL data fixed
PHM data fixed
RL data fixed
ROST data fixed
RCL data fixed
SBUX data fixed
TPR da